# Laboratório Redis - Big Data com Chave-Valor


In [30]:
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import redis
except ImportError:
    install_package("redis")
    import redis

import json
import time
from datetime import datetime, timedelta
import random


## 2. Conexão com Redis Cloud


In [31]:
REDIS_HOST = "redis-14293.c13.us-east-1-3.ec2.redns.redis-cloud.com"
REDIS_PORT = 14293
REDIS_PASSWORD = "ehcS4X87a3rqRPWscbleFH38x0ggYwbW"

# conexxão local
# REDIS_HOST = "localhost"
# REDIS_PORT = 6379
# REDIS_PASSWORD = None

# Criar conexão Redis
try:
    if REDIS_PASSWORD and REDIS_PASSWORD != "your-password":
        redis_client = redis.StrictRedis(
            host=REDIS_HOST,
            port=REDIS_PORT,
            password=REDIS_PASSWORD,
            decode_responses=True
        )
    else:
        # Conexão local para demonstração (sem senha)
        print("Usando conexão Redis local para demonstração...")
        redis_client = redis.StrictRedis(
            host='localhost',
            port=6379,
            decode_responses=True
        )
    

    response = redis_client.ping()
    print(f"Conexão estabelecida! Resposta: {response}")
    
except redis.ConnectionError:
    print("ERRO: Não foi possível conectar ao Redis.")
    print("Verifique se o Redis está rodando ou configure suas credenciais do Redis Cloud.")
    redis_client = None
except Exception as e:
    print(f"Erro inesperado: {e}")
    redis_client = None

Conexão estabelecida! Resposta: True


## 3. Exercícios com Chaves

### Operações Básicas com Chaves Redis

In [32]:
def safe_redis_operation(operation_name, operation_func, *args, **kwargs):
    """executa operação Redis com fallback para simulação"""
    if redis_client:
        try:
            result = operation_func(*args, **kwargs)
            print(f"{operation_name}: {result}")
            return result
        except Exception as e:
            print(f"Erro em {operation_name}: {e}")
            return None
    else:
        print(f"Simulando {operation_name} (Redis não conectado)")
        return f"Simulado: {operation_name}"

print("=== EXERCÍCIOS COM CHAVES ===\n")

# 1. Criar chave "simples" com valor "um valor")
print("1. Criar chave 'simples' com valor 'um valor':")
safe_redis_operation("SET simples", lambda: redis_client.set("simples", "um valor"))
safe_redis_operation("GET simples", lambda: redis_client.get("simples"))

print("\n2. Criar 3 usuários fictícios (usuario:1, usuario:2, usuario:3):")
# 2. Criar 3 usuários fictícios
usuarios = [
    ("usuario:1", "João Silva"),
    ("usuario:2", "Maria Santos"),
    ("usuario:3", "Pedro Costa")
]

for chave, valor in usuarios:
    safe_redis_operation(f"SET {chave}", lambda k=chave, v=valor: redis_client.set(k, v) if redis_client else None)

print("\n3. Listando todas as chaves criadas:")
# 3. Listar todas as chaves
safe_redis_operation("KEYS *", lambda: redis_client.keys("*"))

print("\n4. Listando somente chaves de usuários:")
# 4. Listar chaves de usuários
safe_redis_operation("KEYS usuario:*", lambda: redis_client.keys("usuario:*"))

print("\n5. Criando mais dois usuários (11 e 21):")
# 5. Criar usuários 11 e 21
novos_usuarios = [("usuario:11", "Ana Lima"), ("usuario:21", "Carlos Oliveira")]
for chave, valor in novos_usuarios:
    safe_redis_operation(f"SET {chave}", lambda k=chave, v=valor: redis_client.set(k, v) if redis_client else None)

=== EXERCÍCIOS COM CHAVES ===

1. Criar chave 'simples' com valor 'um valor':
SET simples: True
GET simples: um valor

2. Criar 3 usuários fictícios (usuario:1, usuario:2, usuario:3):
SET usuario:1: True
SET usuario:2: True
SET usuario:3: True

3. Listando todas as chaves criadas:
KEYS *: ['count', 'tarefas', 'usuario:1', 'article:2', 'usuario:2', 'usuario:11', 'tag:tutorial:articles', 'user:2', 'employee:1:reports_to', 'tag:modeling:articles', 'tag:python:articles', 'simples', 'temperature:sensor:1', 'animal', 'equipe:1', 'arvores', 'usuario:31', 'product:1:ratings', 'post:1', 'user:1', 'usuario:21', 'tag:design:articles', 'cesta:1', 'animais', 'article:2:tags', 'product:2', 'tag:nosql:articles', 'product:4', 'article:3:tags', 'employee:2:reports_to', 'cesta:2', 'tag:database:articles', 'article:3', 'allbaskets', 'usuario:01', 'article:1', 'usuario:3', 'tag:redis:articles', 'product:1', 'product:3', 'article:1:tags']

4. Listando somente chaves de usuários:
KEYS usuario:*: ['usuario:1

In [33]:
print("\n6. Listando chaves de usuário terminando em 1:")
# 6. Listar chaves terminando em 1
safe_redis_operation("KEYS usuario:*1", lambda: redis_client.keys("usuario:*1"))

print("\n7. Criar usuário com identificador entre 20-39 terminando em 1:")
# 7. Criar usuário entre 20-39 terminando em 1
safe_redis_operation("SET usuario:31", lambda: redis_client.set("usuario:31", "Fernanda Rocha"))

print("\n8. Listando chaves de usuário terminando em 1 (entre 20-30):")
# 8. Listar chaves terminando em 1 entre 20-30
if redis_client:
    try:
        todas_chaves = redis_client.keys("usuario:*1")
        chaves_filtradas = [k for k in todas_chaves if k.startswith("usuario:2")]
        print(f"Chaves entre 20-30 terminando em 1: {chaves_filtradas}")
    except:
        print("Simulando filtro de chaves")
else:
    print("Simulando filtro de chaves (Redis não conectado)")

print("\n9. Renomeando usuario:1 para usuario:01:")
# 9. Renomear chave
safe_redis_operation("RENAME usuario:1 usuario:01", lambda: redis_client.rename("usuario:1", "usuario:01"))

print("\n10. Listando todas as chaves terminadas em 1:")
# 10. Listar chaves terminadas em 1
safe_redis_operation("KEYS *1", lambda: redis_client.keys("*1"))

print("\n11. Removendo usuário com identificador 3:")
# 11. Remover usuário 3
safe_redis_operation("DEL usuario:3", lambda: redis_client.delete("usuario:3"))

print("\n12. Configurando TTL de 10 segundos para usuario:2:")
# 12. Configurar TTL
safe_redis_operation("EXPIRE usuario:2 10", lambda: redis_client.expire("usuario:2", 10))
safe_redis_operation("TTL usuario:2", lambda: redis_client.ttl("usuario:2"))


6. Listando chaves de usuário terminando em 1:
KEYS usuario:*1: ['usuario:1', 'usuario:11', 'usuario:31', 'usuario:21', 'usuario:01']

7. Criar usuário com identificador entre 20-39 terminando em 1:
SET usuario:31: True

8. Listando chaves de usuário terminando em 1 (entre 20-30):
Chaves entre 20-30 terminando em 1: ['usuario:21']

9. Renomeando usuario:1 para usuario:01:
RENAME usuario:1 usuario:01: True

10. Listando todas as chaves terminadas em 1:
KEYS *1: ['usuario:11', 'temperature:sensor:1', 'equipe:1', 'usuario:31', 'post:1', 'user:1', 'usuario:21', 'cesta:1', 'usuario:01', 'article:1', 'product:1']

11. Removendo usuário com identificador 3:
DEL usuario:3: 1

12. Configurando TTL de 10 segundos para usuario:2:
EXPIRE usuario:2 10: True
TTL usuario:2: 10


10

## 4. Exercícios com Strings


In [34]:
print("=== EXERCÍCIOS COM STRINGS ===\n")

# 1. Criar chave "animal" com valor "gato"
print("1. Criando chave 'animal' com valor 'gato':")
safe_redis_operation("SET animal gato", lambda: redis_client.set("animal", "gato"))

# 2. Trocar valor de "gato" para "peixe" retornando o original
print("\n2. Trocando valor de 'gato' para 'peixe' (GETSET):")
safe_redis_operation("GETSET animal peixe", lambda: redis_client.getset("animal", "peixe"))

# 3. Acrescentar "-espada" ao valor
print("\n3. Acrescentando '-espada' ao valor:")
safe_redis_operation("APPEND animal -espada", lambda: redis_client.append("animal", "-espada"))
safe_redis_operation("GET animal", lambda: redis_client.get("animal"))

# 4. Retornar substring "espada"
print("\n4. Retornando substring 'espada':")
safe_redis_operation("GETRANGE animal 6 11", lambda: redis_client.getrange("animal", 6, 11))

# 5. Retornar substring "peixe"
print("\n5. Retornando substring 'peixe':")
safe_redis_operation("GETRANGE animal 0 4", lambda: redis_client.getrange("animal", 0, 4))

# 6. Criar string "count" com valor 0
print("\n6. Criando string 'count' com valor 0:")
safe_redis_operation("SET count 0", lambda: redis_client.set("count", 0))

# 7. Incrementar "count" em 1
print("\n7. Incrementando 'count' em 1:")
safe_redis_operation("INCR count", lambda: redis_client.incr("count"))

# 8. Incrementar "count" em 10
print("\n8. Incrementando 'count' em 10:")
safe_redis_operation("INCRBY count 10", lambda: redis_client.incrby("count", 10))

# 9. Retornar comprimento da string "count"
print("\n9. Retornando comprimento da string 'count':")
safe_redis_operation("STRLEN count", lambda: redis_client.strlen("count"))

=== EXERCÍCIOS COM STRINGS ===

1. Criando chave 'animal' com valor 'gato':
SET animal gato: True

2. Trocando valor de 'gato' para 'peixe' (GETSET):
GETSET animal peixe: gato

3. Acrescentando '-espada' ao valor:
APPEND animal -espada: 12
GET animal: peixe-espada

4. Retornando substring 'espada':
GETRANGE animal 6 11: espada

5. Retornando substring 'peixe':
GETRANGE animal 0 4: peixe

6. Criando string 'count' com valor 0:
SET count 0: True

7. Incrementando 'count' em 1:
INCR count: 1

8. Incrementando 'count' em 10:
INCRBY count 10: 11

9. Retornando comprimento da string 'count':
STRLEN count: 2


2

## 5. Exercícios com Hashes

Os hashes são estruturas de dados que armazenam mapeamentos entre campos string e valores string.

In [35]:
print("=== EXERCÍCIOS COM HASHES ===\n")

# 1. Verificar se existe a chave "cachorros" na hash "animais"
print("1. Verificando se existe 'cachorros' na hash 'animais':")
safe_redis_operation("HEXISTS animais cachorros", lambda: redis_client.hexists("animais", "cachorros"))

# 2. Criar hash "animais" e adicionar "cachorros" com valor 25
print("\n2. Criando hash 'animais' e adicionando 'cachorros': 25:")
safe_redis_operation("HSET animais cachorros 25", lambda: redis_client.hset("animais", "cachorros", 25))

# 3. Adicionar "gatos" com valor 37
print("\n3. Adicionando 'gatos': 37:")
safe_redis_operation("HSET animais gatos 37", lambda: redis_client.hset("animais", "gatos", 37))

# 4. Adicionar "peixes" com valor 28
print("\n4. Adicionando 'peixes': 28:")
safe_redis_operation("HSET animais peixes 28", lambda: redis_client.hset("animais", "peixes", 28))

# 5. Retornar todo o conteúdo da hash "animais"
print("\n5. Retornando todo o conteúdo da hash 'animais':")
safe_redis_operation("HGETALL animais", lambda: redis_client.hgetall("animais"))

# 6. Retornar valores de "cachorros" e "gatos"
print("\n6. Retornando valores de 'cachorros' e 'gatos':")
safe_redis_operation("HMGET animais cachorros gatos", lambda: redis_client.hmget("animais", ["cachorros", "gatos"]))

# 7. Criar hash "arvores" com múltiplos valores
print("\n7. Criando hash 'arvores' com múltiplos campos:")
arvores_data = {"palmeira": 81, "pinheiro": 23, "baobá": 1}
safe_redis_operation("HMSET arvores", lambda: redis_client.hmset("arvores", arvores_data))

# 8. Mostrar somente valores da hash "arvores"
print("\n8. Mostrando valores da hash 'arvores':")
safe_redis_operation("HVALS arvores", lambda: redis_client.hvals("arvores"))

# 9. Mostrar somente chaves da hash "arvores"
print("\n9. Mostrando chaves da hash 'arvores':")
safe_redis_operation("HKEYS arvores", lambda: redis_client.hkeys("arvores"))

# 10. Corrigir valor de "pinheiro" para 12
print("\n10. Corrigindo valor de 'pinheiro' para 12:")
safe_redis_operation("HSET arvores pinheiro 12", lambda: redis_client.hset("arvores", "pinheiro", 12))
safe_redis_operation("HGET arvores pinheiro", lambda: redis_client.hget("arvores", "pinheiro"))

=== EXERCÍCIOS COM HASHES ===

1. Verificando se existe 'cachorros' na hash 'animais':
HEXISTS animais cachorros: True

2. Criando hash 'animais' e adicionando 'cachorros': 25:
HSET animais cachorros 25: 0

3. Adicionando 'gatos': 37:
HSET animais gatos 37: 0

4. Adicionando 'peixes': 28:
HSET animais peixes 28: 0

5. Retornando todo o conteúdo da hash 'animais':
HGETALL animais: {'cachorros': '25', 'gatos': '37', 'peixes': '28'}

6. Retornando valores de 'cachorros' e 'gatos':
HMGET animais cachorros gatos: ['25', '37']

7. Criando hash 'arvores' com múltiplos campos:


C:\Users\samila.rodrigues\AppData\Local\Temp\ipykernel_57636\1889327276.py:30: DeprecationWarning: Call to deprecated hmset. (Use 'hset' instead.) -- Deprecated since version 4.0.0.
  safe_redis_operation("HMSET arvores", lambda: redis_client.hmset("arvores", arvores_data))


HMSET arvores: True

8. Mostrando valores da hash 'arvores':
HVALS arvores: ['81', '23', '1']

9. Mostrando chaves da hash 'arvores':
HKEYS arvores: ['palmeira', 'pinheiro', 'baobá']

10. Corrigindo valor de 'pinheiro' para 12:
HSET arvores pinheiro 12: 0
HGET arvores pinheiro: 12


'12'

## 6. Exercícios com Listas

As listas são coleções ordenadas de strings, inseridas de acordo com a ordem de inserção.

In [36]:
print("=== EXERCÍCIOS COM LISTAS ===\n")

# 1. Adicionar "Verificar correio" ao final da lista (somente se existir)
print("1. Tentando adicionar 'Verificar correio' com RPUSHX (somente se existir):")
safe_redis_operation("RPUSHX tarefas 'Verificar correio'", lambda: redis_client.rpushx("tarefas", "Verificar correio"))

# 2. Adicionar sem verificar existência
print("\n2. Adicionando 'Verificar correio' com RPUSH (sem verificar):")
safe_redis_operation("RPUSH tarefas 'Verificar correio'", lambda: redis_client.rpush("tarefas", "Verificar correio"))

# 3. Anexar "Abrir correio" ao final
print("\n3. Anexando 'Abrir correio' ao final:")
safe_redis_operation("RPUSH tarefas 'Abrir correio'", lambda: redis_client.rpush("tarefas", "Abrir correio"))

# 4. Adicionar "Iniciar sistema" ao início
print("\n4. Adicionando 'Iniciar sistema' ao início:")
safe_redis_operation("LPUSH tarefas 'Iniciar sistema'", lambda: redis_client.lpush("tarefas", "Iniciar sistema"))

# 5. Obter tamanho da lista
print("\n5. Obtendo tamanho da lista:")
safe_redis_operation("LLEN tarefas", lambda: redis_client.llen("tarefas"))

# 6. Retornar conteúdo de toda a lista
print("\n6. Retornando toda a lista:")
safe_redis_operation("LRANGE tarefas 0 -1", lambda: redis_client.lrange("tarefas", 0, -1))

# 7. Fazer primeira entrada ser "Abrir o correio"
print("\n7. Alterando primeira entrada para 'Abrir o correio':")
safe_redis_operation("LSET tarefas 0 'Abrir o correio'", lambda: redis_client.lset("tarefas", 0, "Abrir o correio"))

# 8. Examinar primeiras 2 entradas
print("\n8. Examinando primeiras 2 entradas:")
safe_redis_operation("LRANGE tarefas 0 1", lambda: redis_client.lrange("tarefas", 0, 1))

# 9. Remover primeira entrada
print("\n9. Removendo primeira entrada:")
safe_redis_operation("LPOP tarefas", lambda: redis_client.lpop("tarefas"))

# 10. Examinar segunda entrada
print("\n10. Examinando segunda entrada (índice 1):")
safe_redis_operation("LINDEX tarefas 1", lambda: redis_client.lindex("tarefas", 1))

# 11. Retornar toda a lista
print("\n11. Retornando toda a lista final:")
safe_redis_operation("LRANGE tarefas 0 -1", lambda: redis_client.lrange("tarefas", 0, -1))

=== EXERCÍCIOS COM LISTAS ===

1. Tentando adicionar 'Verificar correio' com RPUSHX (somente se existir):
RPUSHX tarefas 'Verificar correio': 3

2. Adicionando 'Verificar correio' com RPUSH (sem verificar):
RPUSH tarefas 'Verificar correio': 4

3. Anexando 'Abrir correio' ao final:
RPUSH tarefas 'Abrir correio': 5

4. Adicionando 'Iniciar sistema' ao início:
LPUSH tarefas 'Iniciar sistema': 6

5. Obtendo tamanho da lista:
LLEN tarefas: 6

6. Retornando toda a lista:
LRANGE tarefas 0 -1: ['Iniciar sistema', 'Verificar correio', 'Abrir correio', 'Verificar correio', 'Verificar correio', 'Abrir correio']

7. Alterando primeira entrada para 'Abrir o correio':
LSET tarefas 0 'Abrir o correio': True

8. Examinando primeiras 2 entradas:
LRANGE tarefas 0 1: ['Abrir o correio', 'Verificar correio']

9. Removendo primeira entrada:
LPOP tarefas: Abrir o correio

10. Examinando segunda entrada (índice 1):
LINDEX tarefas 1: Abrir correio

11. Retornando toda a lista final:
LRANGE tarefas 0 -1: ['Ve

['Verificar correio',
 'Abrir correio',
 'Verificar correio',
 'Verificar correio',
 'Abrir correio']

## 7. Exercícios com Sets

Os sets são coleções não ordenadas de strings únicas.

In [37]:
print("=== EXERCÍCIOS COM SETS ===\n")

# 1. Adicionar frutas ao conjunto "cesta:1"
print("1. Adicionando maçãs, laranjas e bananas à cesta:1:")
safe_redis_operation("SADD cesta:1", lambda: redis_client.sadd("cesta:1", "maçãs", "laranjas", "bananas"))

# 2. Listar membros da cesta:1
print("\n2. Listando membros da cesta:1:")
safe_redis_operation("SMEMBERS cesta:1", lambda: redis_client.smembers("cesta:1"))

# 3. Adicionar frutas ao conjunto "cesta:2"
print("\n3. Adicionando abacaxis, bananas e laranjas à cesta:2:")
safe_redis_operation("SADD cesta:2", lambda: redis_client.sadd("cesta:2", "abacaxis", "bananas", "laranjas"))

# 4. Verificar membros da cesta:2
print("\n4. Verificando membros da cesta:2:")
safe_redis_operation("SMEMBERS cesta:2", lambda: redis_client.smembers("cesta:2"))

# 5. Obter interseção dos 2 conjuntos
print("\n5. Obtendo interseção das cestas:")
safe_redis_operation("SINTER cesta:1 cesta:2", lambda: redis_client.sinter("cesta:1", "cesta:2"))

# 6. Mover "abacaxis" de cesta:2 para cesta:1
print("\n6. Movendo 'abacaxis' de cesta:2 para cesta:1:")
safe_redis_operation("SMOVE cesta:2 cesta:1 abacaxis", lambda: redis_client.smove("cesta:2", "cesta:1", "abacaxis"))
print("   Verificando cesta:1 após movimento:")
safe_redis_operation("SMEMBERS cesta:1", lambda: redis_client.smembers("cesta:1"))
print("   Verificando cesta:2 após movimento:")
safe_redis_operation("SMEMBERS cesta:2", lambda: redis_client.smembers("cesta:2"))

# 7. Armazenar união dos conjuntos em "allbaskets"
print("\n7. Armazenando união em 'allbaskets':")
safe_redis_operation("SUNIONSTORE allbaskets cesta:1 cesta:2", lambda: redis_client.sunionstore("allbaskets", "cesta:1", "cesta:2"))
safe_redis_operation("SMEMBERS allbaskets", lambda: redis_client.smembers("allbaskets"))

# 8. Remover "laranjas" de cesta:1
print("\n8. Removendo 'laranjas' de cesta:1:")
safe_redis_operation("SREM cesta:1 laranjas", lambda: redis_client.srem("cesta:1", "laranjas"))
safe_redis_operation("SMEMBERS cesta:1", lambda: redis_client.smembers("cesta:1"))

=== EXERCÍCIOS COM SETS ===

1. Adicionando maçãs, laranjas e bananas à cesta:1:
SADD cesta:1: 1

2. Listando membros da cesta:1:
SMEMBERS cesta:1: {'abacaxis', 'maçãs', 'laranjas', 'bananas'}

3. Adicionando abacaxis, bananas e laranjas à cesta:2:
SADD cesta:2: 1

4. Verificando membros da cesta:2:
SMEMBERS cesta:2: {'abacaxis', 'laranjas', 'bananas'}

5. Obtendo interseção das cestas:
SINTER cesta:1 cesta:2: {'abacaxis', 'laranjas', 'bananas'}

6. Movendo 'abacaxis' de cesta:2 para cesta:1:
SMOVE cesta:2 cesta:1 abacaxis: True
   Verificando cesta:1 após movimento:
SMEMBERS cesta:1: {'abacaxis', 'maçãs', 'laranjas', 'bananas'}
   Verificando cesta:2 após movimento:
SMEMBERS cesta:2: {'laranjas', 'bananas'}

7. Armazenando união em 'allbaskets':
SUNIONSTORE allbaskets cesta:1 cesta:2: 4
SMEMBERS allbaskets: {'abacaxis', 'maçãs', 'laranjas', 'bananas'}

8. Removendo 'laranjas' de cesta:1:
SREM cesta:1 laranjas: 1
SMEMBERS cesta:1: {'abacaxis', 'maçãs', 'bananas'}


{'abacaxis', 'bananas', 'maçãs'}

## 8. Exercícios com Sorted Sets

Os sorted sets são similares aos sets, mas cada elemento tem um score associado que determina a ordem.

In [38]:
print("=== EXERCÍCIOS COM SORTED SETS ===\n")

# 1. Adicionar membros com scores ao conjunto "equipe:1"
print("1. Adicionando membros com scores à equipe:1:")
membros_equipe = {
    "Joao": 69,
    "Leonardo": 67, 
    "Sergio": 70,
    "Sandra": 68,
    "Ana": 70,
    "Maria": 73
}
safe_redis_operation("ZADD equipe:1", lambda: redis_client.zadd("equipe:1", membros_equipe))

# 2. Retornar número de membros
print("\n2. Número de membros no conjunto:")
safe_redis_operation("ZCARD equipe:1", lambda: redis_client.zcard("equipe:1"))

# 3. Contar membros com scores entre 70 e 75
print("\n3. Membros com scores entre 70 e 75:")
safe_redis_operation("ZCOUNT equipe:1 70 75", lambda: redis_client.zcount("equipe:1", 70, 75))

# 4. Obter membros com scores entre 65 e 70
print("\n4. Membros com scores entre 65 e 70:")
safe_redis_operation("ZRANGEBYSCORE equipe:1 65 70", lambda: redis_client.zrangebyscore("equipe:1", 65, 70))

# 5. Retornar classificação da Sandra
print("\n5. Classificação da Sandra:")
safe_redis_operation("ZRANK equipe:1 Sandra", lambda: redis_client.zrank("equipe:1", "Sandra"))

# 6. Membros com scores entre 65-69 em ordem decrescente
print("\n6. Membros (65-69) em ordem decrescente:")
safe_redis_operation("ZREVRANGEBYSCORE equipe:1 69 65", lambda: redis_client.zrevrangebyscore("equipe:1", 69, 65))

=== EXERCÍCIOS COM SORTED SETS ===

1. Adicionando membros com scores à equipe:1:
ZADD equipe:1: 0

2. Número de membros no conjunto:
ZCARD equipe:1: 6

3. Membros com scores entre 70 e 75:
ZCOUNT equipe:1 70 75: 3

4. Membros com scores entre 65 e 70:
ZRANGEBYSCORE equipe:1 65 70: ['Leonardo', 'Sandra', 'Joao', 'Ana', 'Sergio']

5. Classificação da Sandra:
ZRANK equipe:1 Sandra: 1

6. Membros (65-69) em ordem decrescente:
ZREVRANGEBYSCORE equipe:1 69 65: ['Joao', 'Sandra', 'Leonardo']


['Joao', 'Sandra', 'Leonardo']

## 9. Exercícios com TTL (Time To Live)

TTL permite definir quando uma chave deve expirar automaticamente.

In [39]:
print("=== EXERCÍCIOS COM TTL ===\n")

# 1. Adicionar chave "quote:221" com valor "94.23"
print("1. Adicionando chave 'quote:221' com valor '94.23':")
safe_redis_operation("SET quote:221 94.23", lambda: redis_client.set("quote:221", "94.23"))

# 2. Recuperar TTL atual (-1 indica sem TTL configurado)
print("\n2. Verificando TTL atual:")
safe_redis_operation("TTL quote:221", lambda: redis_client.ttl("quote:221"))

# 3. Definir expiração em 30 segundos
print("\n3. Definindo expiração em 30 segundos:")
safe_redis_operation("EXPIRE quote:221 30", lambda: redis_client.expire("quote:221", 30))

# 4. Verificar tempo restante
print("\n4. Verificando tempo restante:")
safe_redis_operation("TTL quote:221", lambda: redis_client.ttl("quote:221"))

# Simulação de monitoramento de TTL
print("\n5. Simulação de monitoramento de TTL:")
if redis_client:
    try:
        for i in range(3):
            ttl = redis_client.ttl("quote:221")
            if ttl == -2:
                print(f"   Tentativa {i+1}: Chave expirou!")
                break
            elif ttl == -1:
                print(f"   Tentativa {i+1}: Chave sem TTL")
                break
            else:
                print(f"   Tentativa {i+1}: TTL restante: {ttl} segundos")
            time.sleep(2)
    except:
        print("    Erro no monitoramento TTL")
else:
    print("    Simulando monitoramento TTL (Redis não conectado)")

print("\nObservação: Em produção, use PERSIST para remover TTL ou EXPIREAT para definir timestamp específico.")

=== EXERCÍCIOS COM TTL ===

1. Adicionando chave 'quote:221' com valor '94.23':
SET quote:221 94.23: True

2. Verificando TTL atual:
TTL quote:221: -1

3. Definindo expiração em 30 segundos:
EXPIRE quote:221 30: True

4. Verificando tempo restante:
TTL quote:221: 30

5. Simulação de monitoramento de TTL:
   Tentativa 1: TTL restante: 30 segundos
   Tentativa 2: TTL restante: 27 segundos
   Tentativa 3: TTL restante: 25 segundos

Observação: Em produção, use PERSIST para remover TTL ou EXPIREAT para definir timestamp específico.


## 10. Modelagem Básica - Artigos e Tags


In [40]:
print("=== MODELAGEM BÁSICA: ARTIGOS E TAGS ===\n")

# Classe para gerenciar artigos e tags
class ArticleTagManager:
    def __init__(self, redis_client):
        self.redis = redis_client
    
    def create_article(self, article_id, name, description, filename, posting_date, tags=None):
        """Criar um artigo com suas tags"""
        if not self.redis:
            print(f" Simulando criação do artigo {article_id}")
            return
        
        # Criar hash do artigo
        article_data = {
            "name": name,
            "description": description, 
            "filename": filename,
            "posting_date": posting_date
        }
        
        self.redis.hmset(f"article:{article_id}", article_data)
        
        # Adicionar tags se fornecidas
        if tags:
            for tag in tags:
                self.add_tag_to_article(article_id, tag)
        
        print(f"  Artigo {article_id} criado: {name}")
    
    def add_tag_to_article(self, article_id, tag):
        """Adicionar tag a um artigo"""
        if not self.redis:
            print(f" Simulando adição da tag '{tag}' ao artigo {article_id}")
            return
        
        # Adicionar artigo à lista de artigos da tag
        self.redis.sadd(f"tag:{tag}:articles", f"article:{article_id}")
        # Adicionar tag à lista de tags do artigo
        self.redis.sadd(f"article:{article_id}:tags", tag)
        
        print(f"  Tag '{tag}' adicionada ao artigo {article_id}")
    
    def list_all_articles(self):
        """Listar todos os artigos"""
        if not self.redis:
            print(" Simulando listagem de artigos")
            return ["article:1", "article:2", "article:3"]
        
        # Buscar todas as chaves de artigos
        article_keys = self.redis.keys("article:*")
        # Filtrar apenas as chaves de hash (não as de tags)
        article_hashes = [key for key in article_keys if not key.endswith(":tags")]
        
        articles = []
        for key in article_hashes:
            article_data = self.redis.hgetall(key)
            if article_data:  # Se é um hash com dados
                article_data['id'] = key
                articles.append(article_data)
        
        return articles
    
    def get_article_with_tags(self, article_id):
        """Obter um artigo com suas tags"""
        if not self.redis:
            print(f"Simulando busca do artigo {article_id}")
            return {"id": f"article:{article_id}", "name": "Artigo Simulado", "tags": ["python", "redis"]}
        
        # Obter dados do artigo
        article_data = self.redis.hgetall(f"article:{article_id}")
        if not article_data:
            return None
        
        # Obter tags do artigo
        tags = self.redis.smembers(f"article:{article_id}:tags")
        
        article_data['id'] = f"article:{article_id}"
        article_data['tags'] = list(tags)
        
        return article_data
    
    def get_articles_by_tag(self, tag):
        """Obter todos os artigos de uma tag"""
        if not self.redis:
            print(f" Simulando busca de artigos da tag '{tag}'")
            return [f"article:1", f"article:3"]
        
        # Obter artigos da tag
        article_ids = self.redis.smembers(f"tag:{tag}:articles")
        
        articles = []
        for article_id in article_ids:
            article_data = self.get_article_with_tags(article_id.split(':')[1])
            if article_data:
                articles.append(article_data)
        
        return articles

# Instanciar o gerenciador
article_manager = ArticleTagManager(redis_client)

print("Criando artigos de exemplo:\n")

# Criar artigos de exemplo
articles_data = [
    (1, "Introdução ao Redis", "Conceitos básicos do Redis", "redis_intro.md", "2024-01-15", ["redis", "database", "nosql"]),
    (2, "Python com Redis", "Como usar Redis em Python", "python_redis.py", "2024-01-20", ["python", "redis", "tutorial"]),
    (3, "Modelagem de Dados", "Padrões de modelagem no Redis", "data_modeling.md", "2024-01-25", ["redis", "modeling", "design"])
]

for article_data in articles_data:
    article_manager.create_article(*article_data)

print(f"\n=== CONSULTAS ===")
print(f"\n1. Listando todos os artigos:")
all_articles = article_manager.list_all_articles()
for article in all_articles:
    print(f"   - {article.get('id', 'N/A')}: {article.get('name', 'N/A')}")

print(f"\n2. Artigo 2 com suas tags:")
article_2 = article_manager.get_article_with_tags(2)
if article_2:
    print(f"   Artigo: {article_2['name']}")
    print(f"   Tags: {', '.join(article_2.get('tags', []))}")

print(f"\n3. Artigos com a tag 'redis':")
redis_articles = article_manager.get_articles_by_tag('redis')
for article in redis_articles:
    print(f"   - {article.get('name', 'N/A')}")

=== MODELAGEM BÁSICA: ARTIGOS E TAGS ===

Criando artigos de exemplo:



C:\Users\samila.rodrigues\AppData\Local\Temp\ipykernel_57636\3197191805.py:22: DeprecationWarning: Call to deprecated hmset. (Use 'hset' instead.) -- Deprecated since version 4.0.0.
  self.redis.hmset(f"article:{article_id}", article_data)


  Tag 'redis' adicionada ao artigo 1
  Tag 'database' adicionada ao artigo 1
  Tag 'nosql' adicionada ao artigo 1
  Artigo 1 criado: Introdução ao Redis
  Tag 'python' adicionada ao artigo 2
  Tag 'redis' adicionada ao artigo 2
  Tag 'tutorial' adicionada ao artigo 2
  Artigo 2 criado: Python com Redis
  Tag 'redis' adicionada ao artigo 3
  Tag 'modeling' adicionada ao artigo 3
  Tag 'design' adicionada ao artigo 3
  Artigo 3 criado: Modelagem de Dados

=== CONSULTAS ===

1. Listando todos os artigos:
   - article:2: Python com Redis
   - article:3: Modelagem de Dados
   - article:1: Introdução ao Redis

2. Artigo 2 com suas tags:
   Artigo: Python com Redis
   Tags: tutorial, python, redis

3. Artigos com a tag 'redis':
   - Python com Redis
   - Modelagem de Dados
   - Introdução ao Redis


## 11. Padrão 1: Embedded Pattern (1:1)


In [41]:
print("=== PADRÃO EMBEDDED (1:1) ===\n")

# Implementação do Padrão Embedded
def embedded_pattern_demo():
    print("Criando produto com padrão embedded:")
    
    product = {
        "name": "Smartphone X",
        "price": 3999,
        "details": {
            "manufacturer": "TechCorp",
            "storage": "128GB", 
            "color": "Black"
        }
    }
    
    # Armazenar produto
    safe_redis_operation("SET product:1", lambda: redis_client.set("product:1", json.dumps(product)))
    
    print(f"\nProduto armazenado: {json.dumps(product, indent=2)}")
    
    # Questão 1: Como recuperar todas as informações do produto?
    print(f"\n1. Recuperando todas as informações:")
    if redis_client:
        try:
            product_data = redis_client.get("product:1")
            if product_data:
                product_obj = json.loads(product_data)
                print(f"   Produto completo: {json.dumps(product_obj, indent=2)}")
            else:
                print("   Produto não encontrado")
        except Exception as e:
            print(f"   Erro: {e}")
    else:
        print("    Simulando: Produto completo recuperado")
    
    # Questão 2: Como acessar apenas o nome e o preço do produto?
    print(f"\n2. Acessando apenas nome e preço:")
    if redis_client:
        try:
            product_data = redis_client.get("product:1")
            if product_data:
                product_obj = json.loads(product_data)
                filtered_data = {
                    "name": product_obj.get("name"),
                    "price": product_obj.get("price")
                }
                print(f"   Dados filtrados: {json.dumps(filtered_data, indent=2)}")
            else:
                print("   Produto não encontrado")
        except Exception as e:
            print(f"   Erro: {e}")
    else:
        print("    Simulando: {\"name\": \"Smartphone X\", \"price\": 3999}")
    
    print(f"\n Vantagens do Padrão Embedded:")
    print(f"   - Acesso rápido a dados relacionados")
    print(f"   - Uma única operação de leitura")
    print(f"   - Estrutura simples")
    print(f"\n Desvantagens:")
    print(f"   - Necessário reescrever todo o objeto para atualizações")
    print(f"   - Pode crescer muito se houver muitos campos")

embedded_pattern_demo()

=== PADRÃO EMBEDDED (1:1) ===

Criando produto com padrão embedded:
SET product:1: True

Produto armazenado: {
  "name": "Smartphone X",
  "price": 3999,
  "details": {
    "manufacturer": "TechCorp",
    "storage": "128GB",
    "color": "Black"
  }
}

1. Recuperando todas as informações:
   Produto completo: {
  "name": "Smartphone X",
  "price": 3999,
  "details": {
    "manufacturer": "TechCorp",
    "storage": "128GB",
    "color": "Black"
  }
}

2. Acessando apenas nome e preço:
   Dados filtrados: {
  "name": "Smartphone X",
  "price": 3999
}

 Vantagens do Padrão Embedded:
   - Acesso rápido a dados relacionados
   - Uma única operação de leitura
   - Estrutura simples

 Desvantagens:
   - Necessário reescrever todo o objeto para atualizações
   - Pode crescer muito se houver muitos campos


## 12. Padrão 2: Partial Embed Pattern (1:N Parcial)


In [42]:
print("=== PADRÃO PARTIAL EMBED (1:N PARCIAL) ===\n")

def partial_embed_pattern_demo():
    print("Criando produto com avaliações recentes embutidas:")
    
    product_with_reviews = {
        "name": "Laptop Ultra",
        "price": 7999,
        "recent_reviews": [
            {"user": "Alice", "rating": 5, "comment": "Excelente desempenho!"},
            {"user": "Bob", "rating": 4, "comment": "Ótimo, mas bateria poderia ser melhor."}
        ]
    }
    
    # Armazenar produto
    safe_redis_operation("SET product:2", lambda: redis_client.set("product:2", json.dumps(product_with_reviews)))
    
    print(f"Produto armazenado: {json.dumps(product_with_reviews, indent=2)}")
    
    # Questão 1: Como acessar apenas as avaliações recentes?
    print(f"\n1. Acessando apenas avaliações recentes:")
    if redis_client:
        try:
            product_data = redis_client.get("product:2")
            if product_data:
                product_obj = json.loads(product_data)
                reviews = product_obj.get("recent_reviews", [])
                print(f"   Avaliações: {json.dumps(reviews, indent=2)}")
            else:
                print("   Produto não encontrado")
        except Exception as e:
            print(f"   Erro: {e}")
    else:
        print("    Simulando: Avaliações recentes recuperadas")
    
    # Questão 2: Como adicionar uma nova avaliação e manter no máximo 3 registros?
    print(f"\n2. Adicionando nova avaliação (máximo 3):")
    
    def add_review_with_limit(product_key, new_review, max_reviews=3):
        if not redis_client:
            print("    Simulando adição de avaliação")
            return
        
        try:
            # Obter produto atual
            product_data = redis_client.get(product_key)
            if product_data:
                product_obj = json.loads(product_data)
                
                # Adicionar nova avaliação no início
                reviews = product_obj.get("recent_reviews", [])
                reviews.insert(0, new_review)
                
                # Manter apenas as últimas N avaliações
                product_obj["recent_reviews"] = reviews[:max_reviews]
                
                # Salvar produto atualizado
                redis_client.set(product_key, json.dumps(product_obj))
                
                print(f"     Avaliação adicionada. Total: {len(product_obj['recent_reviews'])}")
                print(f"   Avaliações atuais: {json.dumps(product_obj['recent_reviews'], indent=2)}")
            else:
                print("   Produto não encontrado")
        except Exception as e:
            print(f"   Erro: {e}")
    
    # Adicionar nova avaliação
    new_review = {"user": "Carlos", "rating": 3, "comment": "Bom custo-benefício"}
    add_review_with_limit("product:2", new_review, 3)
    
    # Adicionar mais uma para testar o limite
    print(f"\n   Adicionando mais uma avaliação para testar limite:")
    another_review = {"user": "Diana", "rating": 5, "comment": "Perfeito para trabalho"}
    add_review_with_limit("product:2", another_review, 3)
    
    print(f"\n Vantagens do Padrão Partial Embed:")
    print(f"   - Acesso rápido aos dados mais relevantes")
    print(f"   - Controle do tamanho do documento") 
    print(f"   - Otimizado para casos de uso específicos")
    print(f"\n Considerações:")
    print(f"   - Requer lógica para manter limites")
    print(f"   - Dados mais antigos podem ser perdidos")

partial_embed_pattern_demo()

=== PADRÃO PARTIAL EMBED (1:N PARCIAL) ===

Criando produto com avaliações recentes embutidas:
SET product:2: True
Produto armazenado: {
  "name": "Laptop Ultra",
  "price": 7999,
  "recent_reviews": [
    {
      "user": "Alice",
      "rating": 5,
      "comment": "Excelente desempenho!"
    },
    {
      "user": "Bob",
      "rating": 4,
      "comment": "\u00d3timo, mas bateria poderia ser melhor."
    }
  ]
}

1. Acessando apenas avaliações recentes:
   Avaliações: [
  {
    "user": "Alice",
    "rating": 5,
    "comment": "Excelente desempenho!"
  },
  {
    "user": "Bob",
    "rating": 4,
    "comment": "\u00d3timo, mas bateria poderia ser melhor."
  }
]

2. Adicionando nova avaliação (máximo 3):
     Avaliação adicionada. Total: 3
   Avaliações atuais: [
  {
    "user": "Carlos",
    "rating": 3,
    "comment": "Bom custo-benef\u00edcio"
  },
  {
    "user": "Alice",
    "rating": 5,
    "comment": "Excelente desempenho!"
  },
  {
    "user": "Bob",
    "rating": 4,
    "comme

## 13-16. Padrões 3-6: Aggregate, Polymorphic, Bucket e Tree/Graph

### Implementação dos Padrões Avançados de Modelagem

In [43]:
print("=== PADRÕES AVANÇADOS DE MODELAGEM ===\n")

# PADRÃO 3: AGGREGATE PATTERN
print("3. PADRÃO AGGREGATE:")
print("   Objetivo: Controle de avaliações agregadas para otimização")

# Criar dados agregados
safe_redis_operation("HMSET product:1:ratings", 
                    lambda: redis_client.hmset("product:1:ratings", {"total_reviews": 2, "sum_ratings": 9}))

# Questão 1: Como calcular a média das avaliações?
if redis_client:
    try:
        ratings_data = redis_client.hgetall("product:1:ratings")
        if ratings_data:
            total = int(ratings_data.get("total_reviews", 0))
            sum_ratings = int(ratings_data.get("sum_ratings", 0))
            average = sum_ratings / total if total > 0 else 0
            print(f"     Média calculada: {average:.1f} ({sum_ratings}/{total})")
        else:
            print("   Dados não encontrados")
    except Exception as e:
        print(f"   Erro: {e}")
else:
    print("    Simulando: Média = 4.5 (9/2)")

# Questão 2: Como atualizar ao adicionar nova avaliação?
print("   Adicionando nova avaliação (rating: 5):")
def update_aggregates(product_id, new_rating):
    if not redis_client:
        print("    Simulando atualização de agregados")
        return
    
    try:
        # Incrementar contadores atomicamente
        redis_client.hincrby(f"product:{product_id}:ratings", "total_reviews", 1)
        redis_client.hincrby(f"product:{product_id}:ratings", "sum_ratings", new_rating)
        
        # Obter nova média
        ratings_data = redis_client.hgetall(f"product:{product_id}:ratings")
        total = int(ratings_data.get("total_reviews", 0))
        sum_ratings = int(ratings_data.get("sum_ratings", 0))
        average = sum_ratings / total if total > 0 else 0
        print(f"     Nova média: {average:.1f} ({sum_ratings}/{total})")
    except Exception as e:
        print(f"   Erro: {e}")

update_aggregates(1, 5)

print("\n4. PADRÃO POLYMORPHIC:")
print("   Objetivo: Coleção de produtos de diferentes categorias")

# Criar produtos de diferentes tipos
products_poly = [
    {"type": "game_console", "name": "GameBox", "storage": "1TB", "hdmi_ports": 2},
    {"type": "earbuds", "name": "SoundBuds", "battery_life": "10h", "connection_type": "Bluetooth"}
]

for i, product in enumerate(products_poly, 3):
    safe_redis_operation(f"SET product:{i}", lambda p=product: redis_client.set(f"product:{i}", json.dumps(p)))

# Questão 1: Como listar todos os produtos?
print("   Listando todos os produtos:")
if redis_client:
    try:
        product_keys = redis_client.keys("product:*")
        # Filtrar apenas produtos (não ratings)
        product_keys = [k for k in product_keys if ":ratings" not in k]
        print(f"     Produtos encontrados: {len(product_keys)}")
        for key in product_keys[:3]:  # Mostrar apenas alguns
            product_data = redis_client.get(key)
            if product_data and product_data.startswith('{'):
                product_obj = json.loads(product_data)
                print(f"     - {key}: {product_obj.get('name', 'N/A')} ({product_obj.get('type', 'N/A')})")
    except Exception as e:
        print(f"   Erro: {e}")
else:
    print("    Simulando: 4 produtos encontrados")

# Questão 2: Como filtrar por tipo?
print("   Filtrando produtos do tipo 'game_console':")
if redis_client:
    try:
        product_keys = redis_client.keys("product:*")
        console_products = []
        for key in product_keys:
            if ":ratings" not in key:
                product_data = redis_client.get(key)
                if product_data and product_data.startswith('{'):
                    product_obj = json.loads(product_data)
                    if product_obj.get('type') == 'game_console':
                        console_products.append(product_obj)
        print(f"     Consoles encontrados: {len(console_products)}")
        for product in console_products:
            print(f"     - {product.get('name', 'N/A')}")
    except Exception as e:
        print(f"   Erro: {e}")
else:
    print("    Simulando: 1 console encontrado (GameBox)")

print("\n5. PADRÃO BUCKET:")
print("   Objetivo: Histórico de temperaturas agrupado em buckets")

# Adicionar medições de temperatura
timestamps_temps = [
    (datetime.now() - timedelta(minutes=i), 22.5 + random.uniform(-2, 2))
    for i in range(5, 0, -1)
]

for timestamp, temp in timestamps_temps:
    safe_redis_operation("ZADD temperature:sensor:1", 
                        lambda ts=timestamp, t=temp: redis_client.zadd("temperature:sensor:1", {str(ts): t}))

# Questão 1: Como listar as últimas 5 medições?
print("   Últimas 5 medições:")
safe_redis_operation("ZREVRANGE temperature:sensor:1 0 4 WITHSCORES", 
                    lambda: redis_client.zrevrange("temperature:sensor:1", 0, 4, withscores=True))

# Questão 2: Como calcular temperatura média?
if redis_client:
    try:
        all_temps = redis_client.zrange("temperature:sensor:1", 0, -1, withscores=True)
        if all_temps:
            temperatures = [score for _, score in all_temps]
            avg_temp = sum(temperatures) / len(temperatures)
            print(f"     Temperatura média: {avg_temp:.1f}°C")
        else:
            print("   Nenhuma medição encontrada")
    except Exception as e:
        print(f"   Erro: {e}")
else:
    print("    Simulando: Temperatura média = 22.8°C")

print("\n6. PADRÃO TREE AND GRAPH:")
print("   Objetivo: Organograma de empresa modelado como grafo")

# Criar relacionamentos hierárquicos
safe_redis_operation("SADD employee:1:reports_to", lambda: redis_client.sadd("employee:1:reports_to", "employee:2", "employee:3"))
safe_redis_operation("SADD employee:2:reports_to", lambda: redis_client.sadd("employee:2:reports_to", "employee:4"))

# Questão 1: Como listar subordinados do funcionário 1?
print("   Subordinados do funcionário 1:")
safe_redis_operation("SMEMBERS employee:1:reports_to", lambda: redis_client.smembers("employee:1:reports_to"))

# Questão 2: Como encontrar gerente do funcionário 4?
print("   Encontrando gerente do funcionário 4:")
if redis_client:
    try:
        # Buscar em todas as relações
        for emp_id in range(1, 5):
            reports = redis_client.smembers(f"employee:{emp_id}:reports_to")
            if "employee:4" in reports:
                print(f"Gerente do funcionário 4: employee:{emp_id}")
                break
        else:
            print("   Gerente não encontrado")
    except Exception as e:
        print(f"   Erro: {e}")
else:
    print("    Simulando: Gerente do funcionário 4 = employee:2")

=== PADRÕES AVANÇADOS DE MODELAGEM ===

3. PADRÃO AGGREGATE:
   Objetivo: Controle de avaliações agregadas para otimização
HMSET product:1:ratings: True


C:\Users\samila.rodrigues\AppData\Local\Temp\ipykernel_57636\3270727516.py:9: DeprecationWarning: Call to deprecated hmset. (Use 'hset' instead.) -- Deprecated since version 4.0.0.
  lambda: redis_client.hmset("product:1:ratings", {"total_reviews": 2, "sum_ratings": 9}))


     Média calculada: 4.5 (9/2)
   Adicionando nova avaliação (rating: 5):
     Nova média: 4.7 (14/3)

4. PADRÃO POLYMORPHIC:
   Objetivo: Coleção de produtos de diferentes categorias
SET product:3: True
SET product:4: True
   Listando todos os produtos:
     Produtos encontrados: 4
     - product:2: Laptop Ultra (N/A)
     - product:4: SoundBuds (earbuds)
     - product:1: Smartphone X (N/A)
   Filtrando produtos do tipo 'game_console':
     Consoles encontrados: 1
     - GameBox

5. PADRÃO BUCKET:
   Objetivo: Histórico de temperaturas agrupado em buckets
ZADD temperature:sensor:1: 1
ZADD temperature:sensor:1: 1
ZADD temperature:sensor:1: 1
ZADD temperature:sensor:1: 1
ZADD temperature:sensor:1: 1
   Últimas 5 medições:
ZREVRANGE temperature:sensor:1 0 4 WITHSCORES: [('2025-08-12 16:36:40.909454', 23.84123171370946), ('2025-08-12 16:37:40.909454', 23.680328953969425), ('2025-08-12 16:14:03.938136', 23.677082791051298), ('2025-08-12 16:38:40.909454', 23.39427439067487), ('2025-08-12 

## 17-18. Padrões 7-8: Revision e Schema Version

### Padrões de Versionamento e Evolução de Esquemas

In [44]:
print("\n7. PADRÃO REVISION:")
print("   Objetivo: Sistema de versões para documentos")

# Criar documento com revisões
post = {
    "title": "Guia Redis",
    "body": "Conteúdo inicial...",
    "revisions": [
        {"title": "Guia Redis", "body": "Conteúdo atualizado 1"},
        {"title": "Guia Redis", "body": "Conteúdo atualizado 2"}
    ]
}

safe_redis_operation("SET post:1", lambda: redis_client.set("post:1", json.dumps(post)))

# Questão 1: Como listar todas as revisões?
print("   Listando revisões do documento:")
if redis_client:
    try:
        post_data = redis_client.get("post:1")
        if post_data:
            post_obj = json.loads(post_data)
            revisions = post_obj.get("revisions", [])
            print(f"Total de revisões: {len(revisions)}")
            for i, revision in enumerate(revisions):
                print(f"     Revisão {i+1}: {revision.get('body', 'N/A')}")
        else:
            print("   Documento não encontrado")
    except Exception as e:
        print(f"   Erro: {e}")
else:
    print("    Simulando: 2 revisões encontradas")

# Questão 2: Como adicionar nova revisão (máximo 5)?
print("   Adicionando nova revisão (máximo 5):")

def add_revision(post_id, new_revision, max_revisions=5):
    if not redis_client:
        print("    Simulando adição de revisão")
        return
    
    try:
        post_data = redis_client.get(f"post:{post_id}")
        if post_data:
            post_obj = json.loads(post_data)
            
            # Adicionar nova revisão no início
            revisions = post_obj.get("revisions", [])
            revisions.insert(0, new_revision)
            
            # Manter apenas as últimas N revisões
            post_obj["revisions"] = revisions[:max_revisions]
            
            # Atualizar corpo principal com a revisão mais recente
            post_obj["body"] = new_revision["body"]
            
            # Salvar documento atualizado
            redis_client.set(f"post:{post_id}", json.dumps(post_obj))
            
            print(f"Revisão adicionada. Total: {len(post_obj['revisions'])}")
        else:
            print("   Documento não encontrado")
    except Exception as e:
        print(f"   Erro: {e}")

new_revision = {"title": "Guia Redis", "body": "Conteúdo atualizado 3 - Padrões avançados"}
add_revision(1, new_revision, 5)

print("\n8. PADRÃO SCHEMA VERSION:")
print("   Objetivo: Esquema evolutivo sem quebrar versões anteriores")

# Criar usuário versão 1
user_v1 = {"schema": "1", "name": "Carlos", "email": "carlos@email.com"}
safe_redis_operation("SET user:1", lambda: redis_client.set("user:1", json.dumps(user_v1)))

# Questão 1: Como modificar esquema para múltiplos emails?
print("   Criando usuário com esquema v2 (múltiplos emails):")
user_v2 = {
    "schema": "2", 
    "name": "Ana", 
    "emails": ["ana@email.com", "ana.work@company.com"]
}
safe_redis_operation("SET user:2", lambda: redis_client.set("user:2", json.dumps(user_v2)))

# Questão 2: Como verificar versão do esquema?
print("   Verificando versões dos esquemas:")

def get_user_with_schema_handling(user_id):
    if not redis_client:
        print(f"    Simulando: user:{user_id} = schema v1")
        return
    
    try:
        user_data = redis_client.get(f"user:{user_id}")
        if user_data:
            user_obj = json.loads(user_data)
            schema_version = user_obj.get("schema", "1")
            
            print(f"user:{user_id} - Schema v{schema_version}")
            
            # Adaptar dados baseado na versão
            if schema_version == "1":
                # Converter email único para lista (compatibilidade)
                email = user_obj.get("email")
                emails = [email] if email else []
                print(f"     Nome: {user_obj.get('name')}, Emails: {emails}")
            elif schema_version == "2":
                emails = user_obj.get("emails", [])
                print(f"     Nome: {user_obj.get('name')}, Emails: {emails}")
                
        else:
            print(f"   Usuário {user_id} não encontrado")
    except Exception as e:
        print(f"   Erro: {e}")

get_user_with_schema_handling(1)
get_user_with_schema_handling(2)




7. PADRÃO REVISION:
   Objetivo: Sistema de versões para documentos
SET post:1: True
   Listando revisões do documento:
Total de revisões: 2
     Revisão 1: Conteúdo atualizado 1
     Revisão 2: Conteúdo atualizado 2
   Adicionando nova revisão (máximo 5):
Revisão adicionada. Total: 3

8. PADRÃO SCHEMA VERSION:
   Objetivo: Esquema evolutivo sem quebrar versões anteriores
SET user:1: True
   Criando usuário com esquema v2 (múltiplos emails):
SET user:2: True
   Verificando versões dos esquemas:
user:1 - Schema v1
     Nome: Carlos, Emails: ['carlos@email.com']
user:2 - Schema v2
     Nome: Ana, Emails: ['ana@email.com', 'ana.work@company.com']
